# Quantum Variational Monte Carlo Ising model in 1D using Stochastic Reconfiguration

In this notebook we will try to find the optimal parameters of a trial wavefunction describing the 1D quantum Ising model with varying number of spins. The trial wavefunction has an RBM-like expression:
$$
    \Psi(\mathcal{S}) = e^{b^Ts}\prod_{i=1}^{n_h}2\cosh(c_i + W^T_is)
$$
where $n_h$ is the number of hidden units, and $\mathbf{\theta} = \{\mathbf{b},\mathbf{c},\textbf{W}\}$ are our variational parameters. According to the variational method, this wavefunction will provide an upper bound to the ground state energy that we are trying to estimate:
$$
    E = \frac{\langle\Psi|\hat{H}|\Psi\rangle}{\langle\Psi|\Psi\rangle}
$$
where the Hamiltonian of the Transverse Field Ising model is:
$$
    \hat{H} = -J\sum_{i=1}^{N}\hat{\sigma}^z_i\hat{\sigma}^z_{i+1} -B\sum_{j=1}^N\hat{\sigma}^x_j
$$
with periodic boundary conditions: $\hat{\sigma}^z_L\hat{\sigma}^z_{L+1} \equiv \hat{\sigma}^z_L\hat{\sigma}^z_1$, and $\hat{\sigma}^z,\hat{\sigma}^x$ being the Pauli matrices acting on the spins, $J,B$ the coupling and the transverse magnetic field term, respectively.

## Statistical Analysis and Autocorrelation Time

Measurements in Markov chains are usually correlated to a certain degree. Though, correlation fades as the number of steps between measured states increases. The distance at which we can consider that two states are uncorrelated is called the **autocorrelation time** $\tau$. However, finding the value analytically is too difficult, so we rely in the binning analysis of the time series to estimate $\tau$.

### Binning Analysis

Assume that we do not have any prior knowledge about the autocorrelation time. Then, we have to use $N_B$ blocks of the time-series samples of increasing lengths $k=2^0,2^1,2^2,\ldots$ until the error estimate for the block converges:
$$
    \varepsilon \approx \sqrt{\frac{s^2_B}{N_B}},\quad\text{where}\quad s^2_B = \frac{1}{N_B-1}\sum_{i=1}^{N_B}(\langle f\rangle_{B_i} - \langle f\rangle_B)^2
$$
The $i$-th block average is computed as
$$
    \langle f\rangle_{B_i} = \frac{1}{k}\sum_{t=1}^kf(x_{(i-1)k+t})
$$
and the mean of all the block averages is
$$
    \langle f\rangle_B = \frac{1}{N_B}\sum_{i=1}\langle f\rangle_{B_i}
$$

The integrated autocorrelation time can be inferred from the binning analysis results. Being
$$
    \tau = \frac{1}{2}\frac{\frac{s^2_B}{N_B}(k\to\infty)}{\frac{s^2_B}{N_B}(k=1)}
$$

## Stochastic Reconfiguration

In order to find the optimal configuration of parameters which minimizes the energy, we will use stochastic reconfiguration of the gradients:
 - Starting from a random initial configuration of the parameters.
 - Proposing a new state $s\to s'$.
 - Accepting the change with probability $|\Psi(s')|^2/|\Psi(s)|^2$.
 - Updating the parameters in our wavefunction as follows:
$$
    \theta^{(t+1)}_k = \theta^{(t)}_k - \delta t\textbf{S}^{-1}\textbf{F}
$$
where we define $\textbf{S}$ as the Fubini-Study metric information matrix
$$
    S_{kk'} = \langle\Delta^*_k\Delta_{k'}\rangle - \langle\Delta^*_k\rangle\langle\Delta_{k'}\rangle
$$
to which we can add an extra regularization term to increase stability, by removing the null diagonal terms $S_{kk} \leftarrow S_{kk} + \lambda$. We also define $\Delta_k$ as the log-derivative of the k-th parameter, and $\textbf{F}$ as the gradient of the mean energy
$$
    F_k = \langle E_{\text{loc}}\Delta^*_k\rangle - \langle E_{\text{loc}}\rangle\langle\Delta^*_k\rangle
$$
To find the ground state of the Hamiltonian, we need to calculate the variational derivatives of the parameters in our RBM ansatz:
$$
    \Delta_{b_j}(s) = \frac{\partial}{\partial b_j}\log(\Psi(s)) = s_j
$$
$$
    \Delta_{c_i}(s) = \frac{\partial}{\partial c_i}\log(\Psi(s)) = \tanh(c_i + \sum_{j=1}^{N}\omega_{ij}s_j)
$$
$$
    \Delta_{\omega_{ij}}(s) = \frac{\partial}{\partial \omega_{ij}}\log(\Psi(s)) = s_j\tanh(c_i + \sum_{j=1}^{N}\omega_{ij}s_j)
$$

## Magnetization of the ground state

We can compare the (longitudinal/transversal) magnetization of the exact ground state with the mean of the sampled state found with Stochastic Reconfiguration. For that, we simply calculate
$$
    m_z = \sqrt{\langle(\hat{\sigma}^z)^2\rangle} = \sqrt{\langle\psi_0|(\tfrac{1}{N}\textstyle\sum_{i=1}^N\hat{\sigma}^z_i)^2|\psi_0\rangle}
$$
$$
    m_x = \sqrt{\langle(\hat{\sigma}^x)^2\rangle} = \sqrt{\langle\psi_0|(\tfrac{1}{N}\textstyle\sum_{i=1}^N\hat{\sigma}^x_i)^2|\psi_0\rangle}
$$
Another way of calculating the exact magnetization, would be to consider large $N$ (or $N\to\infty$), for which the magnetizations simply become:
$$
    m_z = \sqrt{\rho^z_N} = \sqrt{\langle\psi_0|\hat{\sigma}^z_i\hat{\sigma}^z_{i+N}|\psi_0\rangle} = \left(1 - \frac{B^2}{J^2}\right)^{1/8}
$$
if $B < J$, and $0$ otherwise. For the transversal magnetization, we have:
$$
    m_x = \frac{1}{\pi}\int_0^{\pi}\left(\frac{B + J\cos{k}}{\sqrt{J^2 + B^2 + 2JB\cos{k}}}\right)\text{d}k
$$
Now we would only need a way of comparing these values with our found solution with Stochastic Reconfiguration.

When the number of spins is small enough ($N\lesssim20$), we can calculate the projections of our RBM wavefunction onto the spin basis as
$$
    \braket{\mathcal{S}|\Psi_{\mathbf\theta}} = \begin{pmatrix} \Psi_{\mathbf\theta}(s_1) & \Psi_{\mathbf\theta}(s_2) & \ldots & \Psi_{\mathbf\theta}(s_{2^N-1}) & \Psi_{\mathbf\theta}(s_{2^N}) \end{pmatrix}^\intercal
$$

For higher number of spins, the probability density of the ground state can be approximated with a Monte Carlo sampling with the optimized parameters $\mathbf{\theta} = \{\mathbf{b},\mathbf{c},\textbf{W}\}$, given enough samples:
$$
    |\psi_0|^2\sim\frac{1}{N_{\text{samples}}}\sum_{i=1}^{N_{\text{samples}}}\langle\mathcal{S}|\Psi_{\theta}\rangle
$$
Which, can be used to approximate the longitudinal magnetization, since it is diagonal in our spin basis $\mathcal{S}$:
$$
    m_z = \sqrt{\psi'_0\cdot(\sigma^z)^2\cdot\psi_0} = \text{diag}(\sigma^z)\cdot|\psi_0|
$$
but not the transversal magnetization because it is not diagonal in the same spin basis.

### Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from RBM import RBM
import Ising1D as tfi
import SR_Learning as sr

from MagicLoop import magic_function
from multiprocess import Pool

# Set PATH to save txt files
PATH = ''

# Set PATH to save figures
FIG = ''

## RBM Magnetizations vs B/J

The experiment should be done multiple times to calculate the average of the results.
After 10-20 iterations, the results can be plotted below.

If using a HPC cluster, writing a script that calls this file changing the REALIZATION number should be the most efficient approach.

In [ ]:
############### IMPORTANT #################
REALIZATION = 1
###########################################

# List of n values
n_list = [4*i for i in range(1,6)]

# Fix J value
J = 1

# List of B values (concentrated around critical point B/J = 1)
x_list = np.linspace(start=-1., stop=1., num=20)
z_list = (2-0.25)*x_list**3 + 0.25*x_list
B_list = [10.**i for i in z_list]

# Empty magnetization arrays
RBM_mz = np.zeros((len(n_list),len(B_list)))
RBM_mx = np.zeros((len(n_list),len(B_list)))

# Susceptibility
RBM_chix = np.zeros((len(n_list),len(B_list)))

# Empty energies array
RBM_energy = np.zeros((len(n_list),len(B_list)))

max_pool = 10

for (i,n) in enumerate(n_list):    
    # Tuple of arguments
    inputs = list(zip([n]*len(B_list),B_list))
    
    # Inner loop (parallelized)        
    with Pool(max_pool) as p:
        pool_outputs = p.starmap(magic_function,inputs)
    energy, mz, mx, chix, chiz = zip(*pool_outputs)
    RBM_energy[i,:] = np.array(energy)
    RBM_mz[i,:] = np.array(mz)
    RBM_mx[i,:] = np.array(mx)
    RBM_chix[i,:] = np.array(chix)

    print(f"Iteration n: {n} -- Done.\n")

In [ ]:
# Save results
np.savetxt(PATH + f"RBM_mz_{REALIZATION}.txt",RBM_mz)
np.savetxt(PATH + f"RBM_mx_{REALIZATION}.txt",RBM_mx)
np.savetxt(PATH + f"RBM_energy_{REALIZATION}.txt",RBM_energy)
np.savetxt(PATH + f"RBM_chix_{REALIZATION}.txt",RBM_chix)

## Average results

When B/J take extreme values ($10^{-3}$, $10^3$), it may happen that the RBM wavefunction overflows and produces a NaN value. This is due to the fact that, when $N$ is large, the number of parameters explode and even if they are small, the contribution of all of them makes the hyperbolic cosine inside the variational form of the wavefunction overflow.

In [ ]:
# List of n values
n_list = [4*i for i in range(1,6)]

# Fix J value
J = 1

# List of B values (concentrated around critical point B/J = 1)
x_list = np.linspace(start=-1., stop=1., num=20)
z_list = (2-0.25)*x_list**3 + 0.25*x_list
B_list = [10.**i for i in z_list]

# Initialize magnetization arrays
RBM_mz = np.zeros((len(n_list),len(B_list)))
RBM_mx = np.zeros((len(n_list),len(B_list)))
RBM_chix = np.zeros((len(n_list),len(B_list)))
RBM_energy = np.zeros((len(n_list),len(B_list)))

total = 20

for i in range(len(n_list)):
    for j in range(len(B_list)):
        count_mz = 0
        count_mx = 0
        count_chix = 0
        count_energy = 0
        for REALIZATION in range(1,total+1):
            # Load data
            rbm_mz = np.loadtxt(PATH + f"RBM_mz_{REALIZATION}.txt")
            rbm_mx = np.loadtxt(PATH + f"RBM_mx_{REALIZATION}.txt")
            rbm_chix = np.loadtxt(PATH + f"RBM_chix_{REALIZATION}.txt")
            rbm_energy = np.loadtxt(PATH + f"RBM_energy_{REALIZATION}.txt")
            
            # Add value if not NaN
            if not np.isnan(rbm_mz[i,j]):
                count_mz += 1
                RBM_mz[i,j] += rbm_mz[i,j]
                
            if not np.isnan(rbm_mx[i,j]):
                count_mx += 1
                RBM_mx[i,j] += rbm_mx[i,j]

            if not np.isnan(rbm_chix[i,j]):
                count_chix += 1
                RBM_chix[i,j] += rbm_chix[i,j]

            if not np.isnan(rbm_energy[i,j]):
                count_energy += 1
                RBM_energy[i,j] += rbm_energy[i,j]
                
        # Average by total number of not NaNs
        RBM_mz[i,j] /= count_mz
        RBM_mx[i,j] /= count_mx
        RBM_chix[i,j] /= count_chix
        RBM_energy[i,j] /= count_energy

### Longitudinal Mangetization plot

In [ ]:
plt.figure(figsize=(8, 6))
for i in range(len(n_list)):
    plt.plot(B_list, RBM_mz[i,:], marker='o', linestyle='-', label=f'N = {n_list[i]}')

plt.grid(True, linestyle='--', alpha=0.7)
plt.semilogx()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title(r'$\langle m_z\rangle$ of the RBM wavefunction', fontsize=14)
plt.xlabel('B/J', fontsize=12)
plt.ylabel(r'$m_z$', fontsize=12)
plt.legend(fontsize=12)
plt.savefig(FIG + 'RBM_Longitudinal_Magnetizations_avg.png', dpi=300)
plt.show()

### Transversal Mangetization plot

In [ ]:
plt.figure(figsize=(8, 6))
for i in range(len(n_list)):
    plt.plot(B_list, RBM_mx[i,:], marker='o', linestyle='-', label=f'N = {n_list[i]}')

plt.grid(True, linestyle='--', alpha=0.7)
plt.semilogx()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title(r'$\langle m_x\rangle$ of the RBM wavefunction', fontsize=14)
plt.xlabel('B/J', fontsize=12)
plt.ylabel(r'$m_x$', fontsize=12)
plt.legend(fontsize=12)
plt.savefig(FIG + 'RBM_Transversal_Magnetizations_avg.png', dpi=300)
plt.show()

### Magnetic Susceptibility plot

In [ ]:
plt.figure(figsize=(8, 6))
for i in range(len(n_list)):
    plt.plot(B_list, RBM_chix[i,:], marker='o', linestyle='-', label=f'N = {n_list[i]}')

plt.grid(True, linestyle='--', alpha=0.7)
plt.semilogx()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('Magnetic Susceptibility of the RBM wavefunction', fontsize=14)
plt.xlabel('B/J', fontsize=12)
plt.ylabel(r'$m_x$', fontsize=12)
plt.legend(fontsize=12)
plt.savefig(FIG + 'RBM_susceptibility_avg.png', dpi=300)
plt.show()

### Energies plot

In [ ]:
exact_energy = np.zeros((len(n_list),len(B_list)))

for (i,n) in enumerate(n_list):
    for (j,b) in enumerate(B_list):
        exact_energy[i,j] = tfi.groundEnergy(n,1,b)

energy_errors = (exact_energy - RBM_energy)/exact_energy

plt.figure(figsize=(8, 6))
for i in range(len(n_list)):
    plt.plot(B_list, energy_errors[i,:], marker='o', linestyle='-', label=f'N = {n_list[i]}')

plt.grid(True, linestyle='--', alpha=0.7)
plt.semilogx()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('RBM ground state energies', fontsize=14)
plt.xlabel('B/J', fontsize=12)
plt.ylabel(r'energy error', fontsize=12)
plt.legend(fontsize=12)
plt.savefig('RBM_Ground_Energies.png', dpi=300)
plt.show()